In [57]:
# Core libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib as plt
# Sklearn functionality
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression

from sklearn.metrics import accuracy_score

In [58]:
import requests

url = "https://fbref.com/fr/comps/13/Statistiques-Ligue-1#stats_squads_standard_for"
s = requests.get(url).content
stat_equipe = pd.read_html(s)[0]
s = requests.get("https://fbref.com/fr/comps/13/calendrier/Scores-et-tableaux-Ligue-1#sched_2022-2023_13_1").content
matchs = pd.read_html(s)[0]

matchs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 14 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Sem.              380 non-null    float64
 1   Jour              380 non-null    object 
 2   Date              380 non-null    object 
 3   Heure             280 non-null    object 
 4   Domicile          380 non-null    object 
 5   xG                260 non-null    float64
 6   Score             260 non-null    object 
 7   xG.1              260 non-null    float64
 8   Extérieur         380 non-null    object 
 9   Affluence         250 non-null    float64
 10  Tribune           280 non-null    object 
 11  Arbitre           260 non-null    object 
 12  Rapport de match  380 non-null    object 
 13  Notes             0 non-null      float64
dtypes: float64(5), object(9)
memory usage: 45.8+ KB


In [59]:
matchs_trier= matchs.drop(columns=["Sem.","Jour","Tribune","Arbitre","Notes","Rapport de match","Affluence","xG.1", "xG","Heure"],axis=1)
matchs_trier = matchs_trier[matchs["Date"].notna()]
matchs_trier["Date"] = pd.to_datetime(matchs_trier["Date"])
matchs_joue = matchs_trier.dropna()
matchs_joue['Score'] = matchs_joue['Score'].astype('str')
matchs_joue.reset_index(drop=True, inplace=True)
matchs_joue["but_domicile"] = 0
matchs_joue["but_exterieur"] = 0
for i in range(len(matchs_joue)):
    matchs_joue["but_domicile"][i] = matchs_joue["Score"][i][0]
    matchs_joue["but_exterieur"][i] = matchs_joue["Score"][i][2]
matchs_a_jouer = matchs_trier[matchs_trier["Score"].isna()]
matchs_a_jouer["Domicile2"]=matchs_a_jouer["Domicile"]
matchs_a_jouer["Extérieur2"]=matchs_a_jouer["Extérieur"]
matchs_joue.head(15)


C:\Users\jules\AppData\Local\Temp\ipykernel_15616\2067556162.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  matchs_joue['Score'] = matchs_joue['Score'].astype('str')
C:\Users\jules\AppData\Local\Temp\ipykernel_15616\2067556162.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  matchs_joue["but_domicile"] = 0
C:\Users\jules\AppData\Local\Temp\ipykernel_15616\2067556162.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexe

,Date,Domicile,Score,Extérieur,but_domicile,but_exterieur
0,2022-08-05,Lyon,2–1,Ajaccio,2,1
1,2022-08-06,Strasbourg,1–2,Monaco,1,2
2,2022-08-06,Clermont Foot,0–5,Paris S-G,0,5
3,2022-08-07,Toulouse,1–1,Nice,1,1
4,2022-08-07,Lens,3–2,Brest,3,2
5,2022-08-07,Angers,0–0,Nantes,0,0
6,2022-08-07,Lille,4–1,Auxerre,4,1
7,2022-08-07,Montpellier,3–2,Troyes,3,2
8,2022-08-07,Rennes,0–1,Lorient,0,1
9,2022-08-07,Marseille,4–1,Reims,4,1


In [60]:
matchs_joue["but_domicile"] = matchs_joue["but_domicile"].astype(int)
matchs_joue["but_exterieur"] = matchs_joue["but_exterieur"].astype(int)
matchs_joue["resultat_domicile"] = 0
matchs_joue["resultat_exterieur"] = 0
for i in range(len(matchs_joue)):
    if matchs_joue["but_domicile"][i] > matchs_joue["but_exterieur"][i]:
        matchs_joue["resultat_domicile"][i] = "V"
        matchs_joue["resultat_exterieur"][i] = "D"
    elif matchs_joue["but_domicile"][i] < matchs_joue["but_exterieur"][i]:
        matchs_joue["resultat_domicile"][i] = "D"
        matchs_joue["resultat_exterieur"][i] = "V"
    else:
        matchs_joue["resultat_domicile"][i] = "N"
        matchs_joue["resultat_exterieur"][i] = "N"

matchs_joue.head()


C:\Users\jules\AppData\Local\Temp\ipykernel_15616\2063995695.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  matchs_joue["but_domicile"] = matchs_joue["but_domicile"].astype(int)
C:\Users\jules\AppData\Local\Temp\ipykernel_15616\2063995695.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  matchs_joue["but_exterieur"] = matchs_joue["but_exterieur"].astype(int)
C:\Users\jules\AppData\Local\Temp\ipykernel_15616\2063995695.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice fro

,Date,Domicile,Score,Extérieur,but_domicile,but_exterieur,resultat_domicile,resultat_exterieur
0,2022-08-05,Lyon,2–1,Ajaccio,2,1,V,D
1,2022-08-06,Strasbourg,1–2,Monaco,1,2,D,V
2,2022-08-06,Clermont Foot,0–5,Paris S-G,0,5,D,V
3,2022-08-07,Toulouse,1–1,Nice,1,1,N,N
4,2022-08-07,Lens,3–2,Brest,3,2,V,D


In [61]:
matchs_joue.columns

Index(['Date', 'Domicile', 'Score', 'Extérieur', 'but_domicile',
       'but_exterieur', 'resultat_domicile', 'resultat_exterieur'],
      dtype='object')

In [62]:
X = matchs_joue[["Domicile","Extérieur"]]
y = matchs_joue[["but_domicile","but_exterieur","resultat_domicile","resultat_exterieur"]]

In [63]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
cols = ['Domicile', 'Extérieur']
from sklearn import preprocessing
enc = preprocessing.LabelEncoder()

for col in cols:
    X_train[col] = X_train[col].astype('str')
    X_test[col] = X_test[col].astype('str')
    X_train[col] = enc.fit_transform(X_train[col])
    X_test[col] = enc.transform(X_test[col])
    matchs_a_jouer[col] = enc.transform(matchs_a_jouer[col])
    



C:\Users\jules\AppData\Local\Temp\ipykernel_15616\1837660756.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  matchs_a_jouer[col] = enc.transform(matchs_a_jouer[col])
C:\Users\jules\AppData\Local\Temp\ipykernel_15616\1837660756.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  matchs_a_jouer[col] = enc.transform(matchs_a_jouer[col])


In [64]:
ycols = ['but_domicile', 'but_exterieur', 'resultat_domicile', 'resultat_exterieur']
from sklearn.tree import export_text
predictions_dt_liste = []
predictions_lr_liste = []
text = []
for colonne in ycols:
# Build a decision tree model
    model_dt = DecisionTreeClassifier()
    model_dt.fit(X_train, y_train[colonne])
    # Build a logistic regression model
    model_lr = LogisticRegression(solver='lbfgs', multi_class="auto")
    model_lr.fit(X_train, y_train[colonne])
    text.append(export_text(model_dt, feature_names=list(X_train)))
    # Check the model performance with the training data
    predictions_dt = model_dt.predict(X_train)
    print("DecisionTreeClassifier", accuracy_score(y_train[colonne], predictions_dt))
    # Check the model performance with the training data
    predictions_lr = model_lr.predict(X_train)
    print("LogisticRegression", accuracy_score(y_train[colonne], predictions_lr))
    predictions_dt = model_dt.predict(X_test)
    predictions_dt_liste.append( accuracy_score(y_test[colonne], predictions_dt))
    predictions_lr = model_lr.predict(X_test)
    predictions_lr_liste.append(accuracy_score(y_test[colonne], predictions_lr))
    matchs_a_jouer[colonne] = model_lr.predict(matchs_a_jouer[["Domicile","Extérieur"]])
print(predictions_dt_liste)
print(predictions_lr_liste)

c:\Users\jules\miniconda3\envs\myenv\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\jules\AppData\Local\Temp\ipykernel_15616\2706372867.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  matchs_a_jouer[colonne] = model_lr.predict(matchs_a_jouer[["Domicile","Extérieur"]])
c:\Users\jules\miniconda3\envs\

DecisionTreeClassifier 1.0
LogisticRegression 0.35096153846153844
DecisionTreeClassifier 1.0
LogisticRegression 0.40865384615384615
DecisionTreeClassifier 1.0
LogisticRegression 0.4519230769230769
DecisionTreeClassifier 1.0
LogisticRegression 0.4519230769230769
[0.17307692307692307, 0.25, 0.46153846153846156, 0.46153846153846156]
[0.28846153846153844, 0.34615384615384615, 0.5192307692307693, 0.5192307692307693]


C:\Users\jules\AppData\Local\Temp\ipykernel_15616\2706372867.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  matchs_a_jouer[colonne] = model_lr.predict(matchs_a_jouer[["Domicile","Extérieur"]])
C:\Users\jules\AppData\Local\Temp\ipykernel_15616\2706372867.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  matchs_a_jouer[colonne] = model_lr.predict(matchs_a_jouer[["Domicile","Extérieur"]])


In [65]:
matchs_a_jouer["Score"] = matchs_a_jouer["but_domicile"].astype(str) + " - " + matchs_a_jouer["but_exterieur"].astype(str)
matchs_prevu = matchs_a_jouer[["Date","Domicile2","Extérieur2","Score","resultat_domicile"]]
matchs_prevu.head(15)

C:\Users\jules\AppData\Local\Temp\ipykernel_15616\3707043598.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  matchs_a_jouer["Score"] = matchs_a_jouer["but_domicile"].astype(str) + " - " + matchs_a_jouer["but_exterieur"].astype(str)


,Date,Domicile2,Extérieur2,Score,resultat_domicile
287,2023-03-10,Lille,Lyon,1 - 1,D
288,2023-03-11,Auxerre,Rennes,0 - 1,D
289,2023-03-11,Brest,Paris S-G,1 - 1,D
290,2023-03-12,Clermont Foot,Lens,1 - 1,D
291,2023-03-12,Lorient,Troyes,0 - 1,D
292,2023-03-12,Ajaccio,Montpellier,1 - 1,D
293,2023-03-12,Nantes,Nice,1 - 1,V
294,2023-03-12,Angers,Toulouse,0 - 1,D
295,2023-03-12,Monaco,Reims,1 - 1,V
296,2023-03-12,Marseille,Strasbourg,1 - 1,V
